In [1]:
%load_ext lab_black

Step: Load data

In [6]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

In [7]:
# Set up the environment
environment = Environment(name="nfl_injury_scraper")
environment.docker.enabled = True
environment.docker.base_image = None
environment.docker.base_dockerfile = "Dockerfile"

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [13]:
# Create a ScriptRunConfig
src = ScriptRunConfig(
    source_directory=".", script="production_injury_data.py", environment=environment
)

# Create a PythonScriptStep
step = PythonScriptStep(
    name="nfl_injury_scraper",
    script_name="production_injury_data.py",
    compute_target="matthewhanauer991",
    runconfig=src.run_config,
)

# Create a Pipeline
pipeline = Pipeline(workspace=workspace, steps=[step])


# Create an Experiment
experiment = Experiment(workspace=workspace, name="nfl-injury-scraper-exp")

In [14]:
# Publish the Pipeline
published_pipeline = pipeline.publish(name="nfl_injury_scraper_pipeline")

Created step nfl_injury_scraper [6eb1f3ed][4ddefb13-4196-4b78-9a01-08fc4902e973], (This step will run and generate new outputs)


In [ ]:

# Set the schedule recurrence (e.g., every day at 10:00 UTC)
recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="10:00")

In [ ]:




# Create the pipeline schedule
schedule = Schedule.create(workspace=workspace,
                                   name="nfl_injury_scraper_schedule",
                                   pipeline_id=published_pipeline.id,
                                   experiment_name="nfl-injury-scraper-exp",
                                   recurrence=recurrence)
